In [3]:
import game
import time
import copy
import torch
import pygame
import random
import constants
import numpy as np

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
ui = game.GameUI()
ui.update_ui()

In [5]:
g = ui.game

In [ ]:
g.restart()
ui.update_ui()

In [27]:
%%time
while not g.get_is_done():
    g.move(direction=None, inplace=True)
    ui.update_ui()

CPU times: user 1.4 s, sys: 731 ms, total: 2.13 s
Wall time: 7.86 s


In [6]:
def random_ai_move(game_:game.Game, ui: game.GameUI=None, searches_per_move=10, search_length=10):
    scores = np.zeros(4)
    init_game_clone = game_.clone()
    for first_move_i in range(4):
        game_clone1 = init_game_clone.clone()
        if ui is not None:
            ui.set_game(game_clone1)
            ui.update_ui()
        first_move = constants.ARROW_KEYS[first_move_i]
        matrix, score, changed = game_clone1.move(direction=first_move, inplace=True)
        if ui is not None:
            ui.set_game(game_clone1)
            ui.update_ui()
        if changed:
            scores[first_move_i] += score
        else:
            continue
        max_cumulative_score = 0
        for later_moves in range(searches_per_move):
            move_number = 1
            game_clone2 = game_clone1.clone()
            changed = True
            score_cumulative = 0
            while changed and move_number < search_length:
                matrix, score, changed = game_clone2.move(inplace=True) # make a random move
                if ui:
                    ui.set_game(game_clone2)
                    ui.update_ui()
                if changed:
                    score_cumulative += score
                    scores[first_move_i] += score
                    move_number += 1
            max_cumulative_score = max(max_cumulative_score, score_cumulative)
        # scores[first_move_i] += max_cumulative_score
    best_move_index = np.argmax(scores)
    best_move = constants.ARROW_KEYS[best_move_index]
    return best_move

In [7]:
%%time
constants.RANDOM_SEED = 86
g.restart()
ui.update_ui()
for i in range(1500):
    # print(f"i: {i}")
    move = random_ai_move(game_=g, searches_per_move=40, search_length=20)
    g.move(move)
    ui.update_ui()
    if g.get_is_done():
        max_val = g.get_max_val()
        print(f"done, {i} iters, max_val: {max_val}")
        ui.update_ui()
        break
    if g.has_won():
        print(f"won in {i} moves")
        break

done, 722 iters, max_val: 1024
CPU times: user 11min 28s, sys: 15.2 s, total: 11min 44s
Wall time: 11min 36s


In [20]:
pygame.quit()